In [1]:
import os
import json
import random
import collections
import functools
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.io import loadmat
from itertools import count
from utils import getNeighbors, calDist, cir_to_car

from pyecharts.options.global_options import InitOpts
import time
import pyecharts
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.globals import ChartType, SymbolType, GeoType

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False


# delayInGSDir = "./data/StarLink/delay/"
# delayInLeoDir = "/Users/lvmingyang/Seafile/天地一体化-星云CDN/StarLink/delay/"
# neighborDir = "/Users/lvmingyang/Seafile/天地一体化-星云CDN/StarLink/neighbor/"
# interval = 5
# idxSetOfGS = [1584, 1585, 1586, 1587]

# bandwidthOfInterSatellite = 1024 # Mbps
# bandwidthBetweenGS = 200 # Mbps

# neighborOfLeo = loadmat(os.path.join(neighborDir, "1.mat"))['detail']
# # 这里只包含星间的邻接关系

# for i in range(len(neighborOfLeo)):
#     for j in range(len(neighborOfLeo[i])):
#         neighborOfLeo[i][j] -= 1

In [17]:
gs_loc = [[0, 44.7, -120.38],[1, 40.12, -82.40],[2, 25.56, 50.36], [3, 59.20, 18.03], [4, -33.51, 151.12]]
sat_loc_dir = './starlink'

def get_sat_loc(timeslice, dir):
    """
    get sat location info from dir/timeslice.txt
    """
    filename = dir + '/' + str(timeslice)+'.txt'
    with open(filename, 'r') as f:
        lines = f.readlines()
        sat_loc = []
        for line in lines:
            iterms = line.strip().split(',')
            loc = [int(iterms[0]), float(iterms[3]), float(iterms[4]), 530000]
            sat_loc.append(loc)
        # print(sat_loc[0])
        return sat_loc

def load_nxGraph(sat_loc, gs_loc):
    """
    id start from 0
    sat_loc:
        [[sat_0_id, sat_1_lat, sat_1_lon, sat_1_h],
        ......
        [sat_0_id, sat_n_lat, sat_n_lon, sat_n_h]]
    gs_loc:
        [[gs_0_id, gs_1_lat, gs_1_lon],
        ......
        [gs_m_id, gs_m_lat, gs_m_lon]]
    """
    nxGraph = nx.Graph()
    
    # add satellite neighbor edge
    for sat in sat_loc:
        sat_id = sat[0]
        sat_lat = sat[1]
        sat_lon = sat[2]
        sat_h = sat[3]
        neighbors = getNeighbors(sat_id)
        for nei in neighbors:
            nxGraph.add_edge(sat_id, nei, weight=calDist(cir_to_car(sat_lat, sat_lon, sat_h), cir_to_car(sat_loc[nei][1], sat_loc[nei][2], sat_loc[nei][3]))/300000)

    # add satellite-gs edge
    for gs in gs_loc:
        for sat in sat_loc:
            dist = calDist(cir_to_car(sat[1],sat[2],sat[3]), cir_to_car(gs[1], gs[2], 0))
            if dist < 1123000:
                nxGraph.add_edge(sat[0], gs[0]+len(sat_loc), weight=dist/300000)
    
    return nxGraph

def cal_path_delay(path, G):
    totalDelay = 0
    for idx in range(len(path)-1):
        totalDelay += G[path[idx]][path[idx+1]]['weight']
    return totalDelay

os.system("rm path/*")
for timeslice in range(0, 3600*5, 1):
    sat_loc = get_sat_loc(timeslice, 'starlink')
    G = load_nxGraph(sat_loc, gs_loc)
    path_file = 'path/' + str(timeslice) + '.txt'

    with open(path_file, 'w') as f:
        for i in range(len(gs_loc)):   
            path = nx.dijkstra_path(G, 100, i+len(sat_loc))
            # print("from %d to gs %d path is:"%(100, i))
            # print(path)
            delay = nx.dijkstra_path_length(G, 100, i+len(sat_loc))
            # print("path delay is %.2fms"%delay)
            f.writelines("%.2f,%s\n"%(delay, str(path).lstrip('[').rstrip(']')))
    # break
    




[0, -3.369831988593759, 152.418621970556, 530000]
[0, -0.35455183812290914, 154.44231139898238, 530000]
[0, 2.661632512000174, 156.4647850537365, 530000]
[0, 5.673936754385735, 158.49866512440457, 530000]
[0, 8.677485548028985, 160.55681968738287, 530000]
[0, 11.667210465678952, 162.65243100776948, 530000]
[0, 14.63774084421389, 164.79935070920126, 530000]
[0, 17.58328724453857, 167.0121680757453, 530000]
[0, 20.497513812424156, 169.30652424060096, 530000]
[0, 23.37338716057283, 171.69927611063983, 530000]
[0, 26.20301159030405, 174.20872859283753, 530000]
[0, 28.97744296889969, 176.85474387730048, 530000]
[0, 31.68647357864517, 179.6589053618059, 530000]
[0, 34.31838793756662, -177.3554923661108, 530000]
[0, 36.85972374280994, -174.1636630046938, 530000]
[0, 39.29500720081048, -170.7396891737664, 530000]
[0, 41.60651397067204, -167.05746589032083, 530000]
[0, 43.77411548491061, -163.09206296388652, 530000]
[0, 45.77522772303553, -158.8219106570646, 530000]
[0, 47.58501953231922, -154.

KeyboardInterrupt: 

In [63]:
def load_delay_and_neighbor(timeSlice):
    """ 加载星间、星地delay
    Parameters
    ----------
    timeSlice: int
        时间片，范围(1-5731)的整数
    
    Returns
    -------
    delayInLeo: list
        星间delay
    reducedDelayInGS： dict
        星地delay

    For example:
    delayInLeo:
    [[0.         2.1910372  0.         ... 0.         0.         0.        ]
     [2.1910372  0.         2.19019742 ... 0.         0.         0.        ]
     [0.         2.19019742 0.         ... 0.         0.         0.        ]
     ...
     [0.         0.         0.         ... 0.         2.19319721 0.        ]
     [0.         0.         0.         ... 2.19319721 0.         2.19207681]
     [0.         0.         0.         ... 0.         2.19207681 0.        ]]

    reducedDelayInGS:
    {
        1584: {210: 1.9723972613308483, 274: 2.692107219145954, 724: 1.8502827993226718}, 
        1585: {936: 2.526461445765785, 1000: 2.220820621608526, 1001: 2.5596261310866155, 1450: 2.0489803565399685}, 
        1586: {3: 1.880349216318493, 4: 2.689738237732591, 67: 2.23780978442019, 68: 2.409582225748134, 1393: 2.0363002017264917, 1394: 2.450249508896985, 1457: 2.277292782569239, 1458: 2.0864784916218895, 1521: 2.651301677590133, 1522: 2.0240091699230476}, 
        1587: {95: 1.967670116466496, 96: 2.310751909417872, 1167: 2.3324816680995215, 1168: 1.979568262349667}
    }
    """
    
    
    timeSlice = str(timeSlice) + ".mat"
    delayInLeo = loadmat(os.path.join(delayInLeoDir, timeSlice))['delay']
    delayInGS = loadmat(os.path.join(delayInGSDir, timeSlice))['delay']
    
    reducedDelayInGS = {}
    for idxOfGS in idxSetOfGS:
        reducedDelayInGS[idxOfGS] = {}
        for curLeo, curDelay in enumerate(delayInGS[idxOfGS][:1584]):
            if curDelay < 2.71 and curDelay != 0:   
                reducedDelayInGS[idxOfGS][curLeo] = curDelay
    
    return (delayInLeo, reducedDelayInGS)

def convert_link_and_delay_to_nxGraph(delayInLeo, reducedDelayInGS):
    """将邻接关系和delay转换成网络拓扑图
    
    Parameters:
    -----------
    delayInLeo: list
        星间delay
    reducedDelayInGS: dict
        星地delay
        
    Returns:
    --------
    nxGraph: class
        Graph()
        网络拓扑图
    """
    nxGraph = nx.Graph()
    
    for idx in range(1584):
        for nb in neighborOfLeo[idx]:
            nxGraph.add_edge(idx, nb, weight = delayInLeo[idx][nb])
            
#             # 新增内容，增加星地、星间的带宽
#             if idx in idxSetOfGS:
#                 nxGraph.add_edge(idx, nb, weight = delayInLeo[idx][nb], bandwidth = bandwidthBetweenGS)
#             else:
#                 nxGraph.add_edge(idx, nb, weight = delayInLeo[idx][nb], bandwidth = bandwidthOfInterSatellite)
                
#     for idxOfGS in idxSetOfGS:
#         for idx in reducedDelayInGS[idxOfGS].keys():
#             try:
#                 nxGraph.add_edge(idxOfGS, idx, weight = reducedDelayInGS[idxOfGS][idx])
#             except:
#                 print(f"idxOfGS:{idxOfGS}, idxOfLeo:{idx}")
    return nxGraph

def generateBandwidthGraph():
    bwGraph = nx.Graph()
    for idx in range(1584):
        for nb in neighborOfLeo[idx]:
            bwGraph.add_edge(idx, nb, bandwidth = bandwidthOfInterSatellite)
    return bwGraph

def convert_timeSlice_to_nxGraph(timeSlice):
    delayInLeo, reducedDelayInGS = load_delay_and_neighbor(timeSlice)
    nxGraph = convert_link_and_delay_to_nxGraph(delayInLeo, reducedDelayInGS)
    return nxGraph



def  cal_total_delay_according_to_path(path, nxGraph, reducedDelayInGS):
    # return -1 if the path not exist
#     print(path)

#     print(cal_total_delay_according_to_path.__name__)
#         print(f"path[-2]: {path[-2]}, path[-1]:{path[-1]}")
    totalDelay = 0
    for idx in range(len(path)-2):
        totalDelay += nxGraph[path[idx]][path[idx+1]]['weight']
    totalDelay += reducedDelayInGS[path[-1]][path[-2]]
    return totalDelay
    

In [64]:
srcLeo = 1000
interval = 5
maxTimeSlice = 5

paths = []

timeSlice = 1
print(f"current timeSlice: {timeSlice}")
delayInLeo, reducedDelayInGS = load_delay_and_neighbor(timeSlice)
g = convert_link_and_delay_to_nxGraph(delayInLeo, reducedDelayInGS)
lengthAll, pathAll = nx.multi_source_dijkstra(g, {srcLeo})

for curGS in idxSetOfGS:
    visibleLeo = list(reducedDelayInGS[curGS].keys())
            
    for curVisibleLeo in visibleLeo:
        path = pathAll[curVisibleLeo]+[curGS]
        paths.append(path)

print(paths)

current timeSlice: 1
[[1000, 1001, 1002, 1003, 1004, 1005, 1006, 1072, 1138, 1204, 1270, 1336, 1402, 1468, 1534, 16, 82, 148, 214, 215, 216, 217, 218, 219, 220, 221, 1584], [1000, 934, 868, 802, 736, 670, 669, 1584], [1000, 1001, 1002, 1003, 1004, 1005, 1006, 940, 941, 942, 943, 944, 945, 946, 947, 1585], [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1585], [1000, 1066, 1132, 1198, 1264, 1330, 1396, 1395, 1585], [1000, 1066, 1132, 1198, 1264, 1330, 1396, 1585], [1000, 1001, 1002, 1003, 1004, 1070, 1136, 1202, 1268, 1334, 1400, 1466, 1532, 14, 1586], [1000, 1001, 1002, 1003, 1004, 1005, 1071, 1137, 1203, 1269, 1335, 1401, 1467, 1533, 15, 1586], [1000, 1001, 1002, 1068, 1134, 1200, 1266, 1332, 1398, 1464, 1530, 12, 78, 1586], [1000, 1001, 1002, 1003, 1069, 1135, 1201, 1267, 1333, 1399, 1465, 1531, 13, 79, 1586], [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1072, 1138, 1204, 1270, 1336, 1337, 1338, 1339, 1340, 1586], [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1072

In [65]:
# 画路线图
# 正品代码

location = np.load("./data/location.npy")

def randomcolor(num,colored=True):
    colorArr = ['1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
    colorlist=[]
    colorset=set()
    for i in range(num):
        color = ""
        for j in range(6):
            color += colorArr[random.randint(0,14)]
        if color in colorset:
            continue
        colorset.add(color)
        color="#"+color
        if colored:
            colorlist.append({'min':i,'max':i,'label':i,'color':color})
        else:
            colorlist.append({'min':i,'max':i,'label':i,'color':'#868484'})
    return colorlist
def global_config(name):
    '''parameter set {light_speed, r_earth, r_coverage}'''
    config={
        'light_speed':3*10**8,
        'r_earth':6350*10**3,
        'r_coverage':451.352*10**3,
        'planeno':24,
    }
    return config[name]

def get_satno(idx,satnum=66):# idx->[0,1584)
    '''
    Note: for starlink idx in [0,1584]
    INPUT: <satno(idx)>
    OUTPUT: {plane,sat_no}
    '''
    plane=idx//66+1 #[1,24]
    sat_no=(idx+1)%66
    sat_no=66 if sat_no==0 else sat_no
    return [int(plane),int(sat_no)]# [1,24] [1,66]

def plot_sats(location,t=0,src=None,adj=None,des=None,path=None,color=False,path_normal=None,loc_set=None):
    '''
    plot the location of the satellites at the time t
    INPUT: 
        src:[lon,lat]
        adj  :[satno...]
        des  :[lon,lat]
        path :path list
    '''
    # 链式调用
    #%matplotlib inline
    if type(adj)==np.ndarray:# adjacent overhead satellites
        adj=set(map(int,adj))
    else:
        adj=set()
    colorset=randomcolor(global_config('planeno'),colored=color)# 所有轨道颜色
    colorset.append({'min':200,'max':200,'label':'Source','color':'green'}) # 起点颜色
    colorset.append({'min':100,'max':100,'label':'Destination','color':'green'}) # 终点颜色
    c = Geo(init_opts=opts.InitOpts(width='1200px',height='800px'))
    # c.add("", [['Italy','IT']], "world")
    c.add_schema(maptype="world")
    # c.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    #c.set_global_opts(title_opts=opts.TitleOpts(title="加入自定义的点"))
    #data_pair=[]
    for i in range(location.shape[0]):
        plane,no=get_satno(i)
        name='plane:{}, No:{}'.format(plane,no)
        c.add_coordinate(name, location[i,1,t],location[i,0,t]) # lon,lat
        # data_pair.append((name,i//66))
        if i in adj:
            c.add('', [(name,i//66)], type_=GeoType.EFFECT_SCATTER, symbol_size=10)
        else:
            c.add('', [(name,i//66)], type_=GeoType.SCATTER, symbol_size=10)
            
    if src is not None:
        c.add_coordinate('Source (lon:{}, lat:{})'.format(src[0],src[1]), src[0],src[1]) # lon,lat
        c.add('', [('Source (lon:{}, lat:{})'.format(src[0],src[1]),200)], type_=GeoType.EFFECT_SCATTER, symbol_size=10)
    if des is not None:
        c.add_coordinate('Destination (lon:{}, lat:{})'.format(des[0],des[1]), des[0],des[1]) # lon,lat
        c.add('', [('Destination (lon:{}, lat:{})'.format(des[0],des[1]),100)], type_=GeoType.EFFECT_SCATTER, symbol_size=10)
    if loc_set is not None:
        for loc in loc_set:
            c.add_coordinate('Destination (lon:{}, lat:{})'.format(loc[0],loc[1]), loc[0],loc[1]) # lon,lat
            c.add('', [('Destination (lon:{}, lat:{})'.format(loc[0],loc[1]),100)], type_=GeoType.SCATTER, symbol_size=10)
    if path is not None:
        path_traffic = [[0 for i in range(location.shape[0]+4)] for j in range(location.shape[0]+4)]
        for p in path:
            for i in range(len(p)-1):
                path_traffic[p[i]][p[i+1]] = path_traffic[p[i]][p[i+1]] + 1
        print(path_traffic[1000])
        arrow = []
        # for i in range(location.shape[0]):
        #     for j in range(location.shape[0]):
        #         if path_traffic[i][j] > 0:
        #             arrow.append(('plane:{}, No:{}'.format(*get_satno(i)),('plane:{}, No:{}'.format(*get_satno(j)))))
        arrow.append(('plane:{}, No:{}'.format(*get_satno(1000)),('plane:{}, No:{}'.format(*get_satno(1001)))))
        arrow.append(('plane:{}, No:{}'.format(*get_satno(1000)),('plane:{}, No:{}'.format(*get_satno(1001)))))
        c.add("geo",arrow,type_=ChartType.LINES,effect_opts=opts.EffectOpts(symbol=SymbolType.ARROW, symbol_size=6, color='red'),linestyle_opts=opts.LineStyleOpts(curve=0.2,color='red'))
    if path_normal is not None:
        arrow=[('plane:{}, No:{}'.format(*get_satno(path_normal[i])),('plane:{}, No:{}'.format(*get_satno(path_normal[i+1])))) for i in range(len(path_normal)-1)]
        c.add("geo",arrow,type_=ChartType.LINES,effect_opts=opts.EffectOpts(symbol=SymbolType.ARROW, symbol_size=6, color="red"),linestyle_opts=opts.LineStyleOpts(curve=0.2,color='red'))
   
        
    c.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    pieces=colorset
    # is_piecewise 是否自定义分段， 变为true 才能生效
    c.set_global_opts(
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=pieces),
        title_opts=opts.TitleOpts(title="{}".format('')),
    )
    # 在 html(浏览器) 中渲染图表
    c.render(path="index.html")
    # 在 Jupyter Notebook 中渲染图表
    c.render_notebook()

In [66]:
plot_sats(location, path=paths)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [56]:

sourceLeo = 1000
visibleLinkClosed = []
"""
visibleLinkClosed = [
    {
        "start": curStart,
        "end": curEnd,
        "countOfAllLink":countOfAllLink,
        "idxOfGS": curGS,
        "idxOfLeo": curOldLeo,
        "path": curPath,
        "delay": curDelay
    }
]
"""
countOfAllLink = 0
maxNumOfConnInSameTimeSlice = 0
visibleLinkInProgress = {
    1584:{},
    1585:{},
    1586:{},
    1587:{}
}
"""
visibleLinkInProgress = {
    idxOfGS:
         idxOfLeo1: {
            start:1
            end:2
            path:[]
            delay:[]
        },
        idxOfLeo2:{
            ...
        }
}
"""
interval = 5
maxTimeSlice = 5731
for timeSlice in range(1, maxTimeSlice, interval):
    print(f"current timeSlice: {timeSlice}")
    delayInLeo, reducedDelayInGS = load_delay_and_neighbor(timeSlice)
    g = convert_link_and_delay_to_nxGraph(delayInLeo, reducedDelayInGS)
    lengthAll, pathAll = nx.multi_source_dijkstra(g, {sourceLeo})
    for curGS in idxSetOfGS:
        visibleLeoInLastTimeSlice = list(visibleLinkInProgress[curGS].keys())
        visibleLeo = list(reducedDelayInGS[curGS].keys())
        
        for curOldLeo in visibleLeoInLastTimeSlice:
            # 先检测旧的连接是否还存在
            if curOldLeo not in visibleLeo:
                # 说明旧的星地链接不可用了，需要保存数据到visibleLinkClosed
                curStart = visibleLinkInProgress[curGS][curOldLeo]["start"]
                curEnd = visibleLinkInProgress[curGS][curOldLeo]["end"]
                curPath = visibleLinkInProgress[curGS][curOldLeo]["path"]
                curDelay = visibleLinkInProgress[curGS][curOldLeo]["delay"]
                visibleLinkClosed.append({
                    "start": curStart,
                    "end": curEnd,
                    "countOfAllLink":countOfAllLink,
                    "idxOfGS": curGS,
                    "idxOfLeo": curOldLeo,
                    "path": curPath,
                    "delay": curDelay
                })
                countOfAllLink += 1
                
                toStore = np.array(visibleLinkClosed)
                np.save("./visibleLinkClosed.npy", toStore)
                
                del visibleLinkInProgress[curGS][curOldLeo]
                
                
        for curVisibleLeo in visibleLeo:
            if curVisibleLeo not in visibleLinkInProgress[curGS]:
                visibleLinkInProgress[curGS][curVisibleLeo] = {
                    "start": timeSlice,
                    "end": timeSlice,
                    "path": pathAll[curVisibleLeo] + [curGS],
                    "delay": [lengthAll[curVisibleLeo]+reducedDelayInGS[curGS][curVisibleLeo]]
                }
            else:
                visibleLinkInProgress[curGS][curVisibleLeo]["end"] = timeSlice
                curPath = visibleLinkInProgress[curGS][curVisibleLeo]["path"]
                try:
                    curDelay = cal_total_delay_according_to_path(curPath, g, reducedDelayInGS)
                except:
                    print(f"curGS: {curGS}")
                    print(f"curLeo: {curVisibleLeo}")
                    print(f"curPath: {curPath}")
                    print(f"reducedDelayInGS: {reducedDelayInGS}")
                    os._exit()
                          
                visibleLinkInProgress[curGS][curVisibleLeo]["delay"].append(curDelay)

    if timeSlice + interval >= maxTimeSlice:
        # 最后一个时间片，需要将所有可用连接保存到visibleLinkClosed
        for curGS in idxSetOfGS:
            visibleLeoInLastTimeSlice = list(visibleLinkInProgress[curGS])
            visibleLeo = list(reducedDelayInGS[curGS].keys())

            for curOldLeo in visibleLeoInLastTimeSlice:
                curStart = visibleLinkInProgress[curGS][curOldLeo]["start"]
                curEnd = visibleLinkInProgress[curGS][curOldLeo]["end"]
                curPath = visibleLinkInProgress[curGS][curOldLeo]["path"]
                curDelay = visibleLinkInProgress[curGS][curOldLeo]["delay"]
                visibleLinkClosed.append({
                    "start": curStart,
                    "end": curEnd,
                    "countOfAllLink":countOfAllLink,
                    "idxOfGS": curGS,
                    "idxOfLeo": curOldLeo,
                    "path": curPath,
                    "delay": curDelay
                })
                countOfAllLink += 1
                toStore = np.array(visibleLinkClosed)
                np.save("./visibleLinkClosed.npy", toStore)

                del visibleLinkInProgress[curGS][curOldLeo]
                
                


    

current timeSlice: 1
current timeSlice: 6
current timeSlice: 11
current timeSlice: 16
current timeSlice: 21
current timeSlice: 26
current timeSlice: 31
current timeSlice: 36
current timeSlice: 41
current timeSlice: 46
current timeSlice: 51
current timeSlice: 56
current timeSlice: 61
current timeSlice: 66
current timeSlice: 71
current timeSlice: 76
current timeSlice: 81
current timeSlice: 86
current timeSlice: 91
current timeSlice: 96
current timeSlice: 101
current timeSlice: 106
current timeSlice: 111
current timeSlice: 116
current timeSlice: 121
current timeSlice: 126
current timeSlice: 131
current timeSlice: 136
current timeSlice: 141
current timeSlice: 146
current timeSlice: 151
current timeSlice: 156
current timeSlice: 161
current timeSlice: 166
current timeSlice: 171
current timeSlice: 176
current timeSlice: 181
current timeSlice: 186
current timeSlice: 191
current timeSlice: 196
current timeSlice: 201
current timeSlice: 206
current timeSlice: 211
current timeSlice: 216
current ti

KeyboardInterrupt: 